# QIR generation with Q\#

_Tutorials are narrated examples of using QIR as a part of some workflow or with a particular set of tools._

- using the `%qir` magic command in notebooks: https://docs.microsoft.com/en-us/qsharp/api/iqsharp-magic/qir
- use the project file flag to specify QIR compilation of a Q# program: https://github.com/microsoft/qsharp-compiler/tree/main/src/QsCompiler/QirGeneration

In [1]:
// https://github.com/microsoft/Quantum/blob/main/samples/getting-started/teleportation/TeleportationSample.qs
// Copyright (c) Microsoft Corporation.
// Licensed under the MIT License.

open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Random;


operation Teleport (msg : Qubit, target : Qubit) : Unit {
    use register = Qubit();

    H(register);
    CNOT(register, target);

    CNOT(msg, register);
    H(msg);


    if (MResetZ(msg) == One) { Z(target); }

    if (IsResultOne(MResetZ(register))) { X(target); }
}

operation TeleportClassicalMessage (message : Bool) : Bool {
   
    use (msg, target) = (Qubit(), Qubit());

   
    if (message) {
        X(msg);
    }

 
    Teleport(msg, target);

    
    return MResetZ(target) == One;
}

operation TeleportRandomMessage () : Unit {
   
    use (msg, target) = (Qubit(), Qubit());
    PrepareRandomMessage(msg);
    Teleport(msg, target);
    if (MeasureIsPlus(target))  { Message("Received |+>"); }
    if (MeasureIsMinus(target)) { Message("Received |->"); }
    Reset(msg);
    Reset(target);
}

operation PrepareRandomMessage(q: Qubit) : Unit {        
    let choice = DrawRandomBool(0.5);

    if (choice) {
        Message("Sending |−⟩");
        SetToMinus(q);
    } else {
        Message("Sending |+⟩");
        SetToPlus(q);
    }

}

operation SetToPlus(q: Qubit) : Unit {
    Reset(q);
    H(q);
}

operation SetToMinus(q: Qubit) : Unit {
    Reset(q);
    X(q);
    H(q);
}

operation MeasureIsPlus(q: Qubit) : Bool {
    return (Measure([PauliX], [q]) == Zero);
}

operation MeasureIsMinus(q: Qubit) : Bool {
    return (Measure([PauliX], [q]) == One);
}

@EntryPoint()
operation RunProgram () : Unit {
    for idxRun in 1 .. 8 {
        let sent = DrawRandomBool(0.5);
        let received = TeleportClassicalMessage(sent);
        Message($"Round {idxRun}: Sent {sent}, got {received}.");
        Message(sent == received ? "Teleportation successful!" | "");
    }
}


Teleport, TeleportClassicalMessage, TeleportRandomMessage, PrepareRandomMessage, SetToPlus, SetToMinus, MeasureIsPlus, MeasureIsMinus, RunProgram

In [2]:
%simulate TeleportRandomMessage

Sending |−⟩
Received |->


()

In [3]:
%qir RunProgram

Could not load file or assembly 'System.Configuration.ConfigurationManager, Version=6.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51'. Could not find or load a specific file. (0x80131621)
